# Dreams-RNS-CUDA: GPU-Accelerated Ramanujan Dreams Pipeline

This notebook runs the complete Dreams pipeline on an A100 GPU using RNS (Residue Number System) arithmetic for exact integer computation.

## Features
- **Persistent GPU kernel**: Entire walk runs on GPU without CPU round-trips
- **RNS arithmetic**: Exact integer computation with 64+ primes (2000+ bits)
- **Batched execution**: 1000+ shifts processed in parallel
- **Float64 shadow**: Quick delta estimation alongside exact RNS

## Requirements
- Google Colab with A100 GPU runtime
- ~10GB GPU memory for typical runs

## 1. Setup and Installation

In [ ]:
# Check GPU availability
!nvidia-smi
print()

import torch
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
    print("WARNING: No GPU available. Go to Runtime > Change runtime type > GPU")

In [ ]:
# Fix CuPy CUDA version mismatch - IMPORTANT: Run this cell first!
import subprocess
import sys

# Uninstall any existing CuPy versions
!pip uninstall -y cupy cupy-cuda11x cupy-cuda12x cupy-cuda110 cupy-cuda111 cupy-cuda112 cupy-cuda113 cupy-cuda114 cupy-cuda115 cupy-cuda116 cupy-cuda117 cupy-cuda118 cupy-cuda119 cupy-cuda120 cupy-cuda121 cupy-cuda122 cupy-cuda123 cupy-cuda124 2>/dev/null || true

# Detect CUDA version from nvidia-smi (more reliable than nvcc)
try:
    output = subprocess.check_output(['nvidia-smi']).decode()
    # Look for "CUDA Version: XX.Y"
    import re
    match = re.search(r'CUDA Version:\s*(\d+)\.(\d+)', output)
    if match:
        cuda_major = int(match.group(1))
        cuda_minor = int(match.group(2))
        print(f"Detected CUDA {cuda_major}.{cuda_minor}")
    else:
        cuda_major = 12  # Default to 12 for modern Colab
        print(f"Could not detect CUDA version, assuming CUDA 12")
except Exception as e:
    print(f"nvidia-smi failed: {e}, assuming CUDA 12")
    cuda_major = 12

# Install matching CuPy version
if cuda_major >= 12:
    print("Installing cupy-cuda12x...")
    !pip install -q cupy-cuda12x
elif cuda_major == 11:
    print("Installing cupy-cuda11x...")
    !pip install -q cupy-cuda11x
else:
    print(f"Unsupported CUDA {cuda_major}, trying generic cupy...")
    !pip install -q cupy

# Verify CuPy works
try:
    import cupy as cp
    # Force compilation test
    a = cp.array([1, 2, 3])
    b = a * 2
    print(f"CuPy {cp.__version__} installed and working!")
    print(f"Test: {a.get()} * 2 = {b.get()}")
except Exception as e:
    print(f"CuPy verification failed: {e}")
    print("Try restarting the runtime and running this cell again")

# Install other dependencies
!pip install -q sympy mpmath numpy matplotlib

# Clone Dreams-RNS-CUDA (fresh clone to get latest)
!rm -rf Dreams-RNS-CUDA
!git clone https://github.com/VesterlundCoder/Dreams-RNS-CUDA.git

# Clone RNS-CUDA (dependency)
!rm -rf RNS-CUDA
!git clone https://github.com/VesterlundCoder/RNS-CUDA.git

# Add to Python path
sys.path.insert(0, '/content/Dreams-RNS-CUDA/python')
sys.path.insert(0, '/content/RNS-CUDA/python')

print("\n" + "="*50)
print("Setup complete! If CuPy failed, restart runtime and re-run.")
print("="*50)

In [ ]:
# Build RNS-CUDA C++ library (optional - for native CUDA kernels)
%cd /content/RNS-CUDA
!mkdir -p build && cd build && cmake .. -DRNS_ENABLE_GPU=ON -DRNS_ENABLE_TESTS=ON && make -j4

# Run tests to verify
!cd build && ctest --output-on-failure

%cd /content

## 2. Import and Verify

In [ ]:
import numpy as np
import cupy as cp
import math
import time
from typing import List, Dict

# Import Dreams modules
from dreams_rns import compile_cmf, CmfCompiler, Opcode
from dreams_rns import DreamsRunner, WalkConfig
from dreams_rns import analyze_hits, verify_hit
from dreams_rns.compiler import compile_cmf_from_dict, CmfProgram

print(f"CuPy version: {cp.__version__}")
print(f"CUDA available: {cp.cuda.is_available()}")
print(f"Device: {cp.cuda.runtime.getDeviceProperties(0)['name'].decode()}")

## 3. Define CMF Programs

You can define CMFs directly here or load from ramanujantools.

In [ ]:
# Example CMF definitions
CMF_DEFINITIONS = [
    {
        'name': 'Pi_Simple',
        'matrix': {
            (0, 0): 'x',
            (0, 1): '1',
            (1, 0): 'x**2',
            (1, 1): '2*x + 1',
        },
        'm': 2,
        'dim': 1,
        'axis_names': ['x'],
        'directions': [1],
        'target': math.pi
    },
    {
        'name': 'Pi_Advanced',
        'matrix': {
            (0, 0): 'n',
            (0, 1): '1',
            (1, 0): '(2*n + 1)**2',
            (1, 1): '6*n + 5',
        },
        'm': 2,
        'dim': 1,
        'axis_names': ['n'],
        'directions': [1],
        'target': math.pi
    },
    {
        'name': 'Zeta3_Apery',
        'matrix': {
            (0, 0): 'n**3',
            (0, 1): '1',
            (1, 0): '(n + 1)**6',
            (1, 1): '34*n**3 + 51*n**2 + 27*n + 5',
        },
        'm': 2,
        'dim': 1,
        'axis_names': ['n'],
        'directions': [1],
        'target': 1.2020569031595942  # zeta(3)
    },
    {
        'name': 'TwoAxis_4x4',
        'matrix': {
            (0, 0): 'x + y',
            (0, 1): '1',
            (0, 2): '0',
            (0, 3): '0',
            (1, 0): 'x*y',
            (1, 1): 'x - y + 1',
            (1, 2): '1',
            (1, 3): '0',
            (2, 0): 'x**2',
            (2, 1): 'y**2',
            (2, 2): 'x + 1',
            (2, 3): '1',
            (3, 0): '1',
            (3, 1): 'x',
            (3, 2): 'y',
            (3, 3): 'x*y + 1',
        },
        'm': 4,
        'dim': 2,
        'axis_names': ['x', 'y'],
        'directions': [1, 1],
        'target': math.pi
    },
]

print(f"Defined {len(CMF_DEFINITIONS)} CMFs:")
for i, cmf in enumerate(CMF_DEFINITIONS):
    print(f"  {i}: {cmf['name']} ({cmf['m']}x{cmf['m']}, {cmf['dim']}D, target={cmf['target']:.6f})")

In [ ]:
# Compile CMFs to bytecode
programs = []
targets = []

for cmf_def in CMF_DEFINITIONS:
    print(f"Compiling: {cmf_def['name']}...")
    program = compile_cmf_from_dict(
        matrix_dict=cmf_def['matrix'],
        m=cmf_def['m'],
        dim=cmf_def['dim'],
        axis_names=cmf_def['axis_names'],
        directions=cmf_def['directions']
    )
    programs.append(program)
    targets.append(cmf_def['target'])
    print(f"  Opcodes: {len(program.opcodes)}, Constants: {len(program.constants)}")

print(f"\nCompiled {len(programs)} programs")

## 4. Run Dreams Pipeline

In [ ]:
# Configuration for Dreams pipeline
# 
# IMPORTANT: Dreams delta = -(1 + log(|err|) / log(|q|))
# Higher delta = better convergence. Hits are selected where delta > delta_threshold.

CONFIG = WalkConfig(
    K=64,                       # Number of primes (64 * 31 bits = 1984 bits)
    B=1000,                     # Shifts per CMF
    depth=2000,                 # Walk depth
    topk=100,                   # Top-K hits to keep
    target=math.pi,             # Default target (overridden per CMF)
    snapshot_depths=(200, 2000),
    delta_threshold=0.0,        # Min delta for hits (Dreams: higher = better)
    normalize_every=50          # Normalize shadow-float every N steps
)

print("Configuration:")
print(f"  Primes (K): {CONFIG.K}")
print(f"  Batch size (B): {CONFIG.B}")
print(f"  Walk depth: {CONFIG.depth}")
print(f"  Bit capacity: {CONFIG.K * 31} bits")
print(f"  Snapshot depths: {CONFIG.snapshot_depths}")
print(f"  Delta threshold: {CONFIG.delta_threshold} (Dreams: delta > this = hit)")

In [ ]:
# Run pipeline on all CMFs
all_hits = []

for cmf_idx, (program, target, cmf_def) in enumerate(zip(programs, targets, CMF_DEFINITIONS)):
    print(f"\n{'='*60}")
    print(f"CMF {cmf_idx}: {cmf_def['name']}")
    print(f"Target: {target}")
    print(f"{'='*60}")
    
    # Update target in config
    config = WalkConfig(
        K=CONFIG.K,
        B=CONFIG.B,
        depth=CONFIG.depth,
        topk=CONFIG.topk,
        target=target,
        snapshot_depths=CONFIG.snapshot_depths,
        delta_threshold=CONFIG.delta_threshold,
        normalize_every=CONFIG.normalize_every
    )
    
    # Create runner
    runner = DreamsRunner([program], target=target, config=config)
    
    # Time the run
    start = time.time()
    hits = runner.run(
        shifts_per_cmf=config.B,
        depth=config.depth,
        shift_method='random',
        shift_bounds=(-500, 500)
    )
    elapsed = time.time() - start
    
    # Update CMF index in hits
    for hit in hits:
        hit.cmf_idx = cmf_idx
    
    all_hits.extend(hits)
    
    print(f"\nCompleted in {elapsed:.2f}s")
    print(f"Found {len(hits)} hits")
    if hits:
        print(f"Best delta: {hits[0].delta:.2f}")  # Dreams: higher = better

# Sort all hits by delta (highest first - Dreams convention)
all_hits.sort(key=lambda h: h.delta, reverse=True)
print(f"\n{'='*60}")
print(f"Total hits across all CMFs: {len(all_hits)}")

## 5. Analyze Results

In [ ]:
# Analyze all hits
if all_hits:
    results = analyze_hits(all_hits, verbose=True)
else:
    print("No hits found. Try adjusting delta_threshold or increasing depth.")

In [ ]:
# Show top 20 hits
print("Top 20 Hits:")
print("-" * 80)
print(f"{'CMF':<5} {'Shift':<30} {'Depth':<8} {'Delta':<12} {'log(q)':<10}")
print("-" * 80)

for hit in all_hits[:20]:
    shift_str = str(hit.shift)[:28]
    print(f"{hit.cmf_idx:<5} {shift_str:<30} {hit.depth:<8} {hit.delta:<12.2e} {hit.log_q:<10.2f}")

In [ ]:
# Plot results
import matplotlib.pyplot as plt

if all_hits:
    fig, axes = plt.subplots(1, 3, figsize=(15, 4))
    
    # Delta histogram
    deltas = [h.delta for h in all_hits]
    log_deltas = [math.log10(d) if d > 0 else -20 for d in deltas]
    axes[0].hist(log_deltas, bins=30, edgecolor='black', alpha=0.7, color='steelblue')
    axes[0].set_xlabel('log₁₀(Delta)')
    axes[0].set_ylabel('Count')
    axes[0].set_title('Delta Distribution')
    
    # log(q) histogram
    log_qs = [h.log_q for h in all_hits]
    axes[1].hist(log_qs, bins=30, edgecolor='black', alpha=0.7, color='coral')
    axes[1].set_xlabel('log(q)')
    axes[1].set_ylabel('Count')
    axes[1].set_title('log(q) Distribution')
    
    # Delta vs log(q)
    cmf_idxs = [h.cmf_idx for h in all_hits]
    scatter = axes[2].scatter(log_qs, log_deltas, c=cmf_idxs, cmap='viridis', alpha=0.7)
    plt.colorbar(scatter, ax=axes[2], label='CMF Index')
    axes[2].set_xlabel('log(q)')
    axes[2].set_ylabel('log₁₀(Delta)')
    axes[2].set_title('Delta vs log(q)')
    
    plt.tight_layout()
    plt.show()

## 6. Verify Best Hits

In [ ]:
# Verify top hits with high precision
try:
    import mpmath
    mpmath.mp.dps = 50  # 50 decimal places
    
    print("Verifying top 5 hits with high precision:")
    print("-" * 60)
    
    for i, hit in enumerate(all_hits[:5]):
        cmf_def = CMF_DEFINITIONS[hit.cmf_idx]
        program = programs[hit.cmf_idx]
        
        result = verify_hit(hit, program, target=cmf_def['target'], high_precision=True)
        
        print(f"\nHit {i+1}: CMF {hit.cmf_idx} ({cmf_def['name']})")
        print(f"  Shift: {hit.shift}")
        print(f"  Reported delta: {hit.delta:.6e}")
        print(f"  Verified delta: {result['exact_delta']:.6e}")
        print(f"  Match: {'✓' if result['delta_match'] else '✗'}")
        
except ImportError:
    print("mpmath not available for high-precision verification")

## 7. Export Results

In [ ]:
# Export hits to files
from dreams_rns.analysis import export_hits_json, export_hits_csv

if all_hits:
    export_hits_json(all_hits, '/content/dreams_hits.json')
    export_hits_csv(all_hits, '/content/dreams_hits.csv')
    
    # Download links
    from google.colab import files
    print("\nDownload results:")
    # files.download('/content/dreams_hits.json')
    # files.download('/content/dreams_hits.csv')
    print("  Uncomment the download lines above to download files")

## 8. Benchmark

In [ ]:
# Benchmark: measure throughput
print("Benchmarking Dreams pipeline...")
print("="*60)

test_config = WalkConfig(
    K=64,
    B=1000,
    depth=1000,
    topk=10,
    target=math.pi
)

# Use first CMF for benchmark
program = programs[0]
runner = DreamsRunner([program], target=math.pi, config=test_config)

# Warmup
_ = runner.run(shifts_per_cmf=100, depth=100)

# Timed run
start = time.time()
_ = runner.run(shifts_per_cmf=1000, depth=1000)
elapsed = time.time() - start

total_walks = 1000  # shifts
total_steps = 1000 * 1000  # shifts * depth

print(f"\nResults:")
print(f"  Time: {elapsed:.2f}s")
print(f"  Walks/sec: {total_walks / elapsed:.0f}")
print(f"  Steps/sec: {total_steps / elapsed:.0f}")
print(f"  Matrix ops/sec: {total_steps * test_config.K / elapsed:.2e}")

## 9. Custom CMF (Add Your Own)

In [ ]:
# Add your own CMF here
# Example: A custom 2x2 CMF for pi

my_cmf = {
    'name': 'My_Custom_CMF',
    'matrix': {
        (0, 0): 'n + 1',
        (0, 1): '1',
        (1, 0): 'n**2 + n',
        (1, 1): '2*n + 1',
    },
    'm': 2,
    'dim': 1,
    'axis_names': ['n'],
    'directions': [1],
    'target': math.pi
}

# Compile
my_program = compile_cmf_from_dict(
    matrix_dict=my_cmf['matrix'],
    m=my_cmf['m'],
    dim=my_cmf['dim'],
    axis_names=my_cmf['axis_names'],
    directions=my_cmf['directions']
)

print(f"Compiled: {my_cmf['name']}")
print(f"  Opcodes: {len(my_program.opcodes)}")
print(f"  Constants: {len(my_program.constants)}")

# Run
my_runner = DreamsRunner([my_program], target=my_cmf['target'])
my_hits = my_runner.run(shifts_per_cmf=500, depth=1000)

print(f"\nFound {len(my_hits)} hits")
if my_hits:
    print(f"Best hit: delta={my_hits[0].delta:.2e}, shift={my_hits[0].shift}")

## 10. Load from ramanujantools (Optional)

In [ ]:
# Install ramanujantools
!pip install -q ramanujantools

try:
    from ramanujantools import CMF
    import sympy as sp
    
    # Example: Create a CMF from ramanujantools
    x = sp.Symbol('x')
    
    # Define matrix symbolically
    M = sp.Matrix([
        [x, 1],
        [x**2, 2*x + 1]
    ])
    
    # Compile directly from SymPy matrix
    axis_symbols = {'x': 0}
    compiler = CmfCompiler(m=2, dim=1, directions=[1])
    
    for i in range(2):
        for j in range(2):
            if M[i, j] != 0:
                compiler.compile_matrix_entry(i, j, M[i, j], axis_symbols)
    
    rt_program = compiler.build()
    print(f"Compiled from SymPy: {len(rt_program.opcodes)} opcodes")
    
except ImportError:
    print("ramanujantools not available")
except Exception as e:
    print(f"Error: {e}")

## Summary

This notebook demonstrates the Dreams-RNS-CUDA pipeline:

1. **Setup**: Install dependencies and clone repos
2. **Define CMFs**: Create or load CMF definitions
3. **Compile**: Convert CMFs to GPU bytecode
4. **Run**: Execute the walk pipeline on GPU
5. **Analyze**: Examine and verify results
6. **Export**: Save hits for further analysis

### Next Steps
- Try larger batch sizes (B=10000)
- Increase walk depth (depth=5000)
- Add more CMFs from your research
- Adjust delta_threshold for different sensitivity
- Use high-precision verification for best candidates